In [1]:
#! git clone https://github.com/huggingface/transformers.git

remote: Compressing objects: 100% (1974/1974), done.


remote: Total 233244 (delta 24415), reused 23349 (delta 23113), pack-reused 208097 (from 1)
Receiving objects: 100% (233244/233244), 239.05 MiB | 4.10 MiB/s, done.


Resolving deltas: 100% (170753/170753), done.


In [2]:
from pathlib import Path
import pandas as pd


def read_file(path: Path) -> str:
    with open(path, "r") as f:
        return f.read()
    pass

root = Path("./transformers/docs/source/en")
df = pd.DataFrame.from_records([{
    "path": str(path),
    "body": read_file(path),
} for path in root.rglob("*.md") if path.is_file()])

In [3]:
def word_length(text: str) -> int:
    return len(text.split(" "))

In [4]:
df["word_length"] = df["body"].apply(word_length)
df = df.set_index("path")

In [5]:
df = df.drop("transformers/docs/source/en/index.md")

In [6]:
import re
def md_header_split(
        body: str,
        depth: int = 1
):
    exp = re.compile(f"^{'#' * depth} .*$")
    linenums: list[int] = []
    lines = body.split("\n")
    for i, line in enumerate(lines):
        if exp.search(line):
            linenums.append(i)
            pass
        pass

    # each linenum is the start of a new section
    return [
        "\n".join(lines[i:j])
        for i, j in zip(
                [0] + linenums,
                linenums + [len(lines) + 1],
        )
    ]


In [7]:
THRESHOLD_WORDS = 600
THRESHOLD_DEPTH = 4             # give up trying to split on headers at this point


def naive_linebreak_split(text: str) -> list[str]:
    # we just split and group on newlines
    pieces = text.split("\n\n")
    return_pieces = [pieces[0]]
    for piece in pieces[1:]:
        if word_length(return_pieces[-1]) + word_length(piece) <= THRESHOLD_WORDS:
            return_pieces[-1] = "\n\n".join([
                return_pieces[-1],
                piece,
            ])
            pass
        else:
            return_pieces.append(piece)
        pass

    return return_pieces

def recursive_header_split(
        body: str,
        depth: int = 1,
) -> list[str]:
    if word_length(body) <= THRESHOLD_WORDS:
        return [body]

    if depth == THRESHOLD_DEPTH:
        return naive_linebreak_split(body)

    header_splits = md_header_split(body, depth)
    return_splits = []
    for piece in header_splits:
        if word_length(piece) < THRESHOLD_WORDS:
            return_splits.append(piece)
        else:
            return_splits.extend(
                recursive_header_split(piece, depth=depth+1)
            )
        pass
    return return_splits


In [8]:
df["chunks"]= df["body"].apply(recursive_header_split)

In [9]:
chunk_text_by_id = {
    f"{path}:{i}": chunk
     for path, row in df[["chunks"]].iterrows()
    for i, chunk in enumerate(row["chunks"])
}

In [10]:
from transformers import AutoModel

device = "cuda"
embedding_model_id = "jinaai/jina-embeddings-v2-base-en"
embedding_model = AutoModel.from_pretrained(
    embedding_model_id,
    trust_remote_code=True,
).to(device)

/app/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from tqdm.auto import tqdm

batch_size = 1
embeddings = {}
chunk_ids = list(chunk_text_by_id.keys())
chunk_texts = list(chunk_text_by_id.values())

progress_bar = tqdm(range(len(chunk_ids)))
for i in range(0, len(chunk_ids), batch_size):
    batch_embeddings = embedding_model.encode(
        chunk_texts[i:i+batch_size],
        device=device,
    )
    for chunk_id, embedding in zip(
            chunk_ids[i:i+batch_size],
            batch_embeddings
    ):
        embeddings[chunk_id] = embedding
        pass
    progress_bar.update(batch_size)
    pass

100%|████████████████████████████████████████████████████████████████████████| 2883/2883 [01:10<00:00, 39.59it/s]

In [23]:
import json
import faiss
import numpy as np

In [20]:
embedding_size = next(iter(embeddings.values())).shape[-1]
index = faiss.IndexFlatL2(embedding_size)
index.add(np.array(list(embeddings.values())))

In [21]:
faiss.write_index(index, "fragment_index.dat")

In [24]:
with open("./embeddings_order.json", "w") as f:
    f.write(json.dumps(list(embeddings.keys())))
    pass

In [28]:
embeddings_order = list(embeddings.keys())

def search(query: str, k: int):
    query_embed = embedding_model.encode([query], device=device)
    distances, ids = index.search(query_embed, k=k)
    chunk_ids = [
        embeddings_order[idx] for idx in ids[0]
    ]
    return [
        (distance, chunk_id, chunk_text_by_id[chunk_id])
        for distance, chunk_id in zip(distances[0], chunk_ids)
    ]

In [29]:
search("How many layers are in a Phi 3 model?", k=3)

[(82.876724,
  'transformers/docs/source/en/model_doc/phi3.md:0',
  '<!--Copyright 2024 The HuggingFace Team. All rights reserved.\n\nLicensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with\nthe License. You may obtain a copy of the License at\n\nhttp://www.apache.org/licenses/LICENSE-2.0\n\nUnless required by applicable law or agreed to in writing, software distributed under the License is distributed on\nan "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the\nspecific language governing permissions and limitations under the License.\n\n⚠️ Note that this file is in Markdown but contains specific syntax for our doc-builder (similar to MDX) that may not be\nrendered properly in your Markdown viewer.\n\n-->\n\n# Phi-3\n\n## Overview\n\nThe Phi-3 model was proposed in [Phi-3 Technical Report: A Highly Capable Language Model Locally on Your Phone](https://arxiv.org/abs/24

In [37]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

model_id = "microsoft/Phi-3-mini-128k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    ),
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=AutoTokenizer.from_pretrained(model_id),
)


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.08it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
def respond_to_query(
        query: str,
        context: list[str],
) -> str:
    output = pipe([{
        "role": "system",
        "content": "\n".join([
            " ".join([
                "You are a Huggingface Transformers expert.",
                "Your responses may ONLY draw from the following context:",
            ]),
            "---- BEGIN CONTEXT ----",
            *context,
            "---- END CONTEXT ----",
        ])
    }, {
        "role": "user",
        "content": query,
    }], max_new_tokens=1000)
    return output[0]['generated_text'][-1]["content"]

In [26]:
def query_knowledge_assistant(query: str):
    results = search(query, k=3)
    response = respond_to_query(
        query=query,
        context=[
            item[2]
            for item in results
        ]
    )
    print(response)
    return

In [38]:
query_knowledge_assistant("What is the function of the initial embedding layer in a transformer?")

 The initial embedding layer in a transformer, often referred to as the token embedding layer, serves a crucial role in the transformer architecture. Its primary function is to convert the input data, which is typically a sequence of discrete tokens (like words in a text or patches in an image), into a continuous vector representation. This process is essential because transformers, like other deep learning models, operate on numerical data.


In the context of natural language processing (NLP), the input tokens are usually words or subwords, and the token embedding layer transforms these discrete tokens into dense vectors of real numbers. These vectors capture the semantic and syntactic properties of the tokens, enabling the transformer to understand and process the input data effectively.


For image processing tasks, as in the case of ViT (Vision Transformer), the input is a sequence of image patches. Each patch is first converted into a vector using a convolutional layer, which ser

In [39]:
query_knowledge_assistant("How might I implement a novel attention mechanism in the transformers library?")

 To implement a novel attention mechanism in the Transformers library, you would need to follow a series of steps that involve understanding the current attention mechanisms, designing your novel approach, and then integrating it into the Transformers framework. Here's a high-level overview of the process:

1. **Understand Existing Attention Mechanisms**: Familiarize yourself with the attention mechanisms currently available in the Transformers library, such as the original Transformer's self-attention, Scaled Dot-Product Attention (SDPA), and FlashAttention. Study their implementations, strengths, and limitations.

2. **Design Your Novel Attention Mechanism**: Based on your understanding, conceptualize a novel attention mechanism that addresses specific limitations or introduces new capabilities. Consider factors like computational efficiency, memory usage, and adaptability to different tasks.

3. **Implement the Novel Attention Mechanism**:
   - Write the core attention computation c